# Settings for Download: MODIS Vegetation Indeces

[Data](https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MOD13Q1)

In [1]:
# ADJUST ONLY THESE PARAMETERS!

# If loop broke, resume by setting this i to where you want to continue
skip_to_i = 0  # 0 = no skip

# Timescale of Interest
my_plus_years = 5  # Years to add after first visit
my_minus_years = 2  # Years to add before first visit
my_first_date = "-01-01"  # First date of first year "-MM-DD"
my_last_date = "-12-31"  # Last date of last year "-MM-DD"

# Dataset of Interest
product = "MODIS/061/MOD13Q1"
product_start_date = "2000-02-18"
product_end_date = "2023-09-30"
product_scale = 250

output_scale = product_scale
output_folder = "gee-raw-data/modis-vi"

my_bands = ["NDVI", "EVI", "DetailedQA", "SummaryQA"]

In [2]:
# Import Libraries
import pandas as pd
from datetime import datetime
import numpy as np
import sys

sys.path.insert(0, "../../src")

# Import Functions
from run_mp import *
from gee_data_wrangling import *
from gee_data_download import *

# Load data from csv
data = get_location_site_df()

# Clean first and last date for download routine
data_clean = adjust_first_last_date(
    df=data,
    plus_years=my_plus_years,
    minus_years=my_minus_years,
    first_date=my_first_date,
    last_date=my_last_date,
)

# Split sites into groups of ten, to use all cores
sequence = np.arange(1, 11)  # This creates an array [1, 2, ..., 10]
repeated_sequence = np.tile(sequence, len(data_clean) // len(sequence) + 1)[
    : len(data_clean)
]
data_clean["group_id"] = repeated_sequence

# Create a list of all the groups
grouped = data_clean.groupby("group_id")
df_list = [group for name, group in grouped]

# Parallelize the download
run_mp(
    download_gee_data_PARALLEL,
    df_list,
    combine_func=pd.concat,
    progress_bar=True,
    num_cores=10,
    product=product,
    my_bands=my_bands,
    product_start_date=product_start_date,
    product_end_date=product_end_date,
    product_scale=product_scale,
    output_scale=output_scale,
    output_folder=output_folder,
)

100%|██████████| 10/10 [1:18:18<00:00, 469.81s/it]   


ValueError: All objects passed were None